In [ ]:
#Import Libraries
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import scipy.stats as ss
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Visualization Settings
%matplotlib inline
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)  #This displays all columns in the dataframe
pd.set_option('display.float_format', '{:,.2f}'.format)  # Display describe() with normal float formatting
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
#Load the dataset
df=pd.read_csv("../data/raw/Melbourne_housing_FULL.csv")

In [ ]:
#First 5 rows of the dataset
df.head()

In [ ]:
#Structure of the dataset
rows=df.shape[0]
columns=df.shape[1]
print("Number of rows:", rows )
print("Number of columns:", columns)

In [ ]:
#Data Types & Missing Counts
print(df.info())

In [ ]:
# Convert 'Date' column to datetime object
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Verification
df.info()

In [ ]:
#Duplicate check(rows)
duplicates=df.duplicated().sum()
print("Number of duplicate rows:", duplicates)

# Show duplicated rows
df[df.duplicated()]

#Remove duplicates
df=df.drop_duplicates(keep='first').reset_index(drop=True)

#Verify
print(f"Duplicates after: {df.duplicated().sum()}")

In [ ]:
#Sanity check
display(df.describe().T)

In [ ]:
#Check how many impossible BuildingArea values are there (<5.0)
print(f"Count of Impossible BuildingArea values: {len(df[df['BuildingArea'] < 5])}")

# The Fix: Replace values < 5.0 with NaN (Missing) 
df.loc[df['BuildingArea'] < 5, 'BuildingArea'] = np.nan

print(f"Count of Impossible BuildingArea values after fix:{len(df[df['BuildingArea'] < 5])}")

In [ ]:
#Get the dynamic current year
current_year=datetime.datetime.now().year

#Check count of invalid years before fix
print(f"Invalid YearBuilt entries before fix: {((df['YearBuilt'] < 1800) | (df['YearBuilt'] > current_year)).sum()}")

#The fix - replace with NaN
df['YearBuilt'] = df['YearBuilt'].where(df['YearBuilt'].between(1800,current_year), np.nan)

# 3. Verify
print(f"Invalid YearBuilt entries after fix: {((df['YearBuilt'] < 1800) | (df['YearBuilt'] > current_year)).sum()}")

In [ ]:
#Check count of 0 in bathroom
count_bathroom=(df['Bathroom']==0).sum()
print(f"Count of 0 in Bathroom before :{count_bathroom}")

#The fix - replace 0 with NaN
df['Bathroom']=df['Bathroom'].replace(0, np.nan)
print(f"Count of 0 in Bathroom after :{(df['Bathroom']==0).sum()}")

In [ ]:
#Inconsistency Scan
text_cols_to_check = df.select_dtypes(include=['object']).columns
for col in text_cols_to_check:
    if col in df.columns:
        unique_count = df[col].nunique()
        print(f"\n[{col}] Unique Values ({unique_count}):")
        print(sorted(df[col].unique().astype(str)))
        

In [ ]:
#Inconsistency Fix

#Suburb: fix casing
df['Suburb'] = df['Suburb'].str.title()

#SellerG: remove branch info & fix casing
df['SellerG'] = df['SellerG'].str.split('/').str[0].str.strip().str.title()

#Verification
print("--- Suburb ---")
print(sorted(df[df['Suburb'].str.contains('Croydon', case=False)]['Suburb'].unique()))
print(sorted(df[df['Suburb'].str.contains('Viewbank', case=False)]['Suburb'].unique()))

print("\n--- SellerG ---")
brands_to_check = ['Buxton', 'Hockingstuart', 'Vicprop']
for brand in brands_to_check:
    print(f"{brand} variations: {sorted([s for s in df['SellerG'].unique() if brand.lower() in s.lower()])}")

In [ ]:
#Missingness Analysis
msno.matrix(df, figsize=(12, 6), sparkline=False);
plt.show()

In [ ]:
# Calculate number of missing values per column
missing_count = df.isnull().sum()
missing_count = missing_count.sort_values(ascending=False)

print("Count of Missing Values\n")
print(missing_count)

In [ ]:
#Drop rows where 'Price' is missing (Target)
df=df.dropna(subset=['Price'])

#Drop rows where 'Postcode' is missing
df=df.dropna(subset=['Postcode'])

#Verify
print(df.isnull().sum().sort_values(ascending=False))

In [ ]:
#Negative/Zero Check of numeric columns

# 1. Select numeric columns (exclude Lat/Long)
numeric_cols = df.select_dtypes(include=np.number).columns
cols_to_check = [col for col in numeric_cols if col not in ['Lattitude', 'Longtitude']]

print(f"--- Checking {len(cols_to_check)} numeric columns for Non-Positive values ---")


for col in cols_to_check:
    # Count Negatives 
    neg_count = (df[col] < 0).sum()
    
    # Count Zeros 
    zero_count = (df[col] == 0).sum()
    
    if neg_count > 0 or zero_count > 0:
        print(f"{col:<15} | Negatives: {neg_count:<5} | Zeros: {zero_count}")

In [ ]:
#Temporal Analysis (Time Series)

#Extract Year-Month for grouping
df['YearMonth'] = df['Date'].dt.to_period('M')

print("📅 GENERATING TIME SERIES PLOT...")
print("   Goal: Check if the market crashed (which affects how we split the data).")

#Aggregate Average Price per Month
#Count how many houses sold per month too, to make sure the average is reliable
monthly_stats = df.groupby('YearMonth').agg(
    Average_Price=('Price', 'mean'),
    Count=('Price', 'count')
).reset_index()

#Convert back to string for plotting
monthly_stats['YearMonth'] = monthly_stats['YearMonth'].astype(str)

#Plot
plt.figure(figsize=(14, 6))
sns.lineplot(data=monthly_stats, x='YearMonth', y='Average_Price', marker='o', linewidth=2, color='royalblue')

plt.title('Melbourne Housing Market: Price Trend Over Time')
plt.xticks(rotation=45)
plt.xlabel('Month')
plt.ylabel('Average Price ($)')
plt.grid(True, linestyle='--', alpha=0.5)


plt.tight_layout()
plt.show()

In [ ]:
#Data Splitting
#Define Features (X) and Target (y)
X = df.drop('Price', axis=1)
y = df['Price']

#Random Split
#shuffle=True (ensures mix 2016, 2017, and 2018 data together)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

#Verify
print("--- Random Split Successful ---")
print(f"Training Set: {X_train.shape[0]} rows")
print(f"Test Set:     {X_test.shape[0]} rows")

#Safety Check
assert len(X_train) + len(X_test) == len(df), "Error: Row mismatch!"
print("Safety Check Passed.")

In [ ]:
#Cardinality Check
categorical_cols = X_train.select_dtypes(include=['object']).columns

print("\nCardinality Check (Count of Unique Values)")
cardinality = X_train[categorical_cols].nunique().sort_values(ascending=False)
print(cardinality)


In [ ]:
#Target Variable Analysis
plt.figure(figsize=(14, 5))

# Plot Histogram (Distribution)
sns.histplot(y_train.dropna(), kde=True, color='blue')
plt.title('Price Distribution')


plt.tight_layout()
plt.show()



In [ ]:
#Target Variable Analysis
plt.figure(figsize=(14, 5))

# Plot Q-Q Plot (Check Normality)
ss.probplot(y_train.dropna(), dist="norm", plot=plt)
plt.title('Q-Q Plot (Normality Check)')

plt.tight_layout()
plt.show()



In [ ]:
#Target Variable Analysis
plt.figure(figsize=(14, 5))

# Plot Boxplot (Outliers)
sns.boxplot(x=y_train, color='cyan')
plt.title('Price Boxplot')

plt.tight_layout()
plt.show()


In [ ]:
# Calculate Skewness & Kurtosis
price_skew = y_train.skew()
price_kurt = y_train.kurt()

print("Target Statistics")
print(f"Skewness: {price_skew:.4f} (Rule: > 1.0 needs Log Transform)")
print(f"Kurtosis: {price_kurt:.4f}")

In [ ]:
#Data Integrity Check

# List of features that should be integers
discrete_cols = ['Rooms', 'Bedroom2', 'Bathroom', 'Car', 'Postcode', 'YearBuilt', 'Propertycount']

# Check if any value has a non-zero decimal part
print("Checking for non-integer values (decimals)...\n")

for col in discrete_cols:
    if col in X_train.columns:
        # We drop NA first because NaN is technically a float
        has_decimals = (X_train[col].dropna() % 1 != 0).any()
        
        if has_decimals:
            print(f"ALERT: '{col}' contains decimals! ")
            examples = X_train[col][X_train[col] % 1 != 0].head(3).tolist()
            print(f"Examples: {examples}")
        else:
            print(f"'{col}' is clean (Integers only).")

In [ ]:
#Numeric Feature Distribution

#Select all numeric columns
all_numeric = X_train.select_dtypes(include=['float64', 'int64']).columns.tolist()

#Filter out columns don't need
drop_for_plot = ['Price', 'Postcode',] 
cols_to_plot = [c for c in all_numeric if c not in drop_for_plot]

print(f"Scanning {len(cols_to_plot)} Numeric Features...\n")

for col in cols_to_plot:
    unique_count = X_train[col].nunique()
    skew_val = X_train[col].skew()
    
    #Calculate IQR outlier count
    Q1, Q3 = X_train[col].quantile([0.25, 0.75])
    IQR = Q3 - Q1
    outlier_count = ((X_train[col] < Q1 - 1.5*IQR) | (X_train[col] > Q3 + 1.5*IQR)).sum()
    
    plt.figure(figsize=(12, 4))
    
    #Discrete / Count Variables
    if unique_count < 25:
        plt.subplot(1, 2, 1)
        sns.histplot(X_train[col].dropna(), discrete=True, kde=True, color='purple')
        plt.title(f'{col} (Discrete) | Skew: {skew_val:.2f}')
        plt.xlabel(col)
        plt.ylabel('Count')
        
        plt.subplot(1, 2, 2)
        sns.boxplot(x=X_train[col].dropna(), color='cyan')
        plt.title(f'{col} Outliers ({outlier_count})')
        
        #Stats summary for discrete
        stats_msg = f"[{col}] Mode: {X_train[col].mode()[0]} | Skew: {skew_val:.4f} | Outliers: {outlier_count}"

    #Continuous Variables
    else:
        plt.subplot(1, 2, 1)
        sns.histplot(X_train[col].dropna(), kde=True, bins=30, color='blue')
        plt.title(f'{col} (Continuous) | Skew: {skew_val:.2f}')
        
        plt.subplot(1, 2, 2)
        sns.boxplot(x=X_train[col].dropna(), color='orange')
        plt.title(f'{col} Outliers ({outlier_count})')
        
        #Stats summary for continuous
        stats_msg = f"[{col}] Min: {X_train[col].min()} | Max: {X_train[col].max()} | Skew: {skew_val:.4f} | Outliers: {outlier_count}"
    plt.tight_layout()
    plt.show()
    print(stats_msg)
    print("-" * 60)

In [ ]:
#Categorical Feature Distributions

# Select Categorical Columns
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()

#Exclude unwanted features
cols_to_ignore = ['Address'] 
cat_cols = [col for col in cat_cols if col not in cols_to_ignore]

#Define High Cardinality Threshold
#If a column has > 50 unique values, plotting it is messy.
high_cardinality_cutoff = 50 

print(f"Scanning {len(cat_cols)} Categorical Features\n")

for col in cat_cols:
    unique_count = X_train[col].nunique()
    
    #High cardinality scenario
    if unique_count > high_cardinality_cutoff:
        print(f"ANALYZING: {col}")
        print(f"{col} has {unique_count} unique values (Too many to plot).")
        
        # Calculate percentages
        top_5_series = X_train[col].value_counts(normalize=True).head(5) * 100
        
        #Format each number individually to include '%'
        print("Top 5 Categories:",top_5_series.apply(lambda x: f"{x:.2f}%").to_string())
        
        #Check for rare labels
        freq = X_train[col].value_counts(normalize=True) * 100
        rare_count = len(freq[freq < 1.0])
        print(f"Contains {rare_count} rare categories (<1%) that will need encoding.")
        print("-" * 60)
        continue

    #Low cardinality scenario
    print(f"ANALYZING: {col}")
    print(f"Unique Categories: {unique_count}")

    #check for rare labels
    freq = X_train[col].value_counts(normalize=True) * 100
    rare_cats = freq[freq < 1.0] # Threshold: 1%
    
    if not rare_cats.empty:
        print(f"Rare Labels Found (<1%): {len(rare_cats)}")
        print(f"Examples: {rare_cats.index.tolist()}") # Show first 5
    else:
        print("No Rare Labels found.")

    #Plot
    plt.figure(figsize=(10, 6) if unique_count > 10 else (8, 5))
    
    #Use Horizontal Bars if there are many categories
    if unique_count > 10:
        sns.countplot(y=X_train[col], order=X_train[col].value_counts().index, palette="viridis")
        plt.title(f'{col} Distribution (Horizontal View)')
    else:
        sns.countplot(x=X_train[col], order=X_train[col].value_counts().index, palette="viridis")
        plt.title(f'{col} Distribution')
        plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()
    print("-" * 60)

In [ ]:
#Correlation & Multicollinearity Check

#Select only numeric columns for correlation analysis
numeric_df = X_train.select_dtypes(include=['number']).copy()
numeric_df = numeric_df.drop(columns=['Price'], errors='ignore')

#Correlation Heatmap
plt.figure(figsize=(12, 8))
corr_matrix = numeric_df.corr(numeric_only=True)

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

sns.heatmap(corr_matrix, mask=mask, annot=True, fmt=".2f", cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap (Numeric Features)')
plt.show()

#VIF
print("\n🔹 CALCULATING VIF (Multicollinearity Check)...")
print("   Rule: VIF > 5 is suspicious. VIF > 10 is definitely redundant.\n")

#Drop NaNs
vif_data = numeric_df.dropna()

#Add constant
vif_data = add_constant(vif_data)

vif_df = pd.DataFrame()
vif_df["Feature"] = vif_data.columns
vif_df["VIF_Score"] = [
    variance_inflation_factor(vif_data.values, i)
    for i in range(vif_data.shape[1])
]

#Drop const row
vif_df = vif_df[vif_df["Feature"] != "const"]

#Show sorted output
print(vif_df.sort_values(by="VIF_Score", ascending=False).to_string(index=False))

In [ ]:
#Categorical vs Categorical (Redundancy Check)

#Define Categorical Columns to Check
#Avoid extremely high-cardinality columns
available_cols = X_train.columns.tolist()
target_cols = ['Suburb', 'Type', 'Method', 'Regionname', 'CouncilArea']

#
cat_features = [col for col in target_cols if col in available_cols]

#Cramer's V Function
def cramers_v(x, y):
    #Case 1: Same column → Perfect association
    if x.name == y.name:
        return 1.0

    #Case 2: A column with only 1 unique category → No association possible
    if x.nunique() < 2 or y.nunique() < 2:
        return 0.0

    #Contingency table
    confusion_matrix = pd.crosstab(x, y)

    #Chi-square
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape

    #Bias correction
    with np.errstate(divide='ignore', invalid='ignore'):
        phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
        rcorr = r - ((r - 1)**2) / (n - 1)
        kcorr = k - ((k - 1)**2) / (n - 1)

        denom = min((kcorr - 1), (rcorr - 1))
        if denom <= 0:
            return 0.0
        return np.sqrt(phi2corr / denom)

#Calculate Cramer's V Matrix
print("CALCULATING CATEGORICAL ASSOCIATIONS (Cramer's V)...")
print("   Goal: Find redundant categorical features (Score > 0.80 means redundancy).\n")

rows = []
for var1 in cat_features:
    col_values = []
    for var2 in cat_features:
        v = cramers_v(X_train[var1], X_train[var2])
        col_values.append(v)
    rows.append(col_values)

cramers_df = pd.DataFrame(rows, columns=cat_features, index=cat_features)
cramers_df.fillna(0, inplace=True)

#Plot the Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cramers_df, annot=True, fmt=".2f", cmap='Greens', vmin=0, vmax=1)
plt.title("Categorical Association Heatmap (Cramer's V)")
plt.show()

# Print Table
print("\nCramer's V Association Table:")
print(cramers_df)

#Identify Highly Associated Pairs
threshold = 0.8
print("\nHighly Associated Categorical Pairs (Cramer's V > 0.8):")
for i in range(len(cat_features)):
    for j in range(i+1, len(cat_features)):
        if cramers_df.iloc[i, j] > threshold:
            print(f"{cat_features[i]} ↔ {cat_features[j]} : {cramers_df.iloc[i, j]:.2f}")


In [ ]:
#Numeric vs Target (relationship between the target and numerical features)

#Create a temp dataframe combining X and y for plotting
#Use a copy so it don't accidentally modify the real X_train
plot_df = X_train.copy()
plot_df['Price'] = y_train

#Numeric features to check
features_to_plot = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt', 'Car', 'Propertycount']

print(f"Generating Individual Scatter Plots for {len(features_to_plot)} features vs Price...\n")

for col in features_to_plot:
    #Check if column exists
    if col not in plot_df.columns: continue

    plt.figure(figsize=(10, 6))
    
    #Plot with hue='Type' to see if Houses (h) behave differently than Units (u)
    sns.scatterplot(
        data=plot_df.sample(n=min(len(plot_df), 2000), random_state=42), #Sample 2000 points for speed/clarity
        x=col, 
        y='Price', 
        hue='Type', 
        alpha=0.6, 
        palette='Set1'
    )
    
    plt.title(f"Price vs {col}")
    plt.xlabel(col)
    plt.ylabel("Price")
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print("-" * 80)

In [ ]:
#Categorical vs Target (relationship between the target and categorical features)

#Define the categorical features to check
#Stick to the manageable ones (Suburb/Council are too big to plot here)
plot_df = X_train.copy()
plot_df['Price'] = y_train

cat_cols = ['Type', 'Method', 'Regionname'] 
target = 'Price'

print("GENERATING BOXPLOTS...")
print("Goal: Look for categories that separate the price (boxes at different heights).\n")

for col in cat_cols:
    if col not in plot_df.columns: continue

    plt.figure(figsize=(12, 6))
    
    #Sort the order by Median Price so the chart is easy to read (Low -> High)
    order = plot_df.groupby(col)[target].median().sort_values().index
    
    #Create the boxplot
    sns.boxplot(data=plot_df, x=col, y=target, order=order, palette='coolwarm')
    
    plt.title(f'{col} vs Price Distribution')
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, axis='y', alpha=0.3)
    plt.ylabel('Price')
    plt.xlabel(col)
    plt.show()


In [ ]:
#Numeric vs Categorical (Redundancy Check)

#Select all numeric columns
numeric_cols = X_train.select_dtypes(include=['int64', 'float64']).columns

#Select only Low cardinality categorical columns
categorical_cols = [col for col in X_train.select_dtypes(include=['object', 'category']).columns 
                    if X_train[col].nunique() < 20]

print(f"Analyzing redundancy between:")
print(f"Numeric: {list(numeric_cols)}")
print(f"Categorical: {categorical_cols}")

#Prepare a temporary dataframe
X_check = X_train.copy()

#Encode categorical to numbers (MI requires numbers)
for col in categorical_cols:
    X_check[col] = LabelEncoder().fit_transform(X_check[col].astype(str))

#Impute missing numeric values (MI cannot handle NaNs)
for col in numeric_cols:
    X_check[col] = X_check[col].fillna(X_check[col].median())

#Compute Mutual Information
mi_results = pd.DataFrame(index=numeric_cols, columns=categorical_cols, dtype=float)

for cat_col in categorical_cols:
    mi = mutual_info_classif(
        X_check[numeric_cols],   # Predictors(The Numerical features)
        X_check[cat_col],        # Target(The Categorical feature)
        discrete_features=False,
        random_state=42
    )
    mi_results[cat_col] = mi

#Interpret Results
threshold = 0.15 

redundant_pairs = []
for num in numeric_cols:
    for cat in categorical_cols:
        score = mi_results.loc[num, cat]
        if score > threshold:
            redundant_pairs.append((num, cat, score))

#Sort by strongest redundancy first
redundant_pairs.sort(key=lambda x: x[2], reverse=True)

print(f"\nAnalysis Complete. Found {len(redundant_pairs)} potential redundancies:")
for num, cat, score in redundant_pairs:
    print(f"{num} ↔ {cat} : MI Score = {score:.3f}")

#Plot
plt.figure(figsize=(10, 6))
sns.heatmap(mi_results, annot=True, cmap='Blues', fmt=".2f")
plt.title("Mutual Information: Which Numeric Features Predict Categorical Features?")
plt.xlabel("Categorical Features")
plt.ylabel("Numeric Features")
plt.tight_layout()
plt.show()